In [ ]:
Download pdfs
https://maint.loc.gov/law/help/artificial-intelligence/regulation-artificial-intelligence.pdf

read this pdf to text using pymupdf4llm

using instructor and google ai studio models, generate document and question pairs
1. simple per page questions
2. complex questions based on a few pages

Now evaluete this on lance db with 
fts
vector
hybrid
fts+ reranker
vector + reranker
hybrid + reranker
following with ragatouille
colbert -small
colbert jina 

metrics to evaluate
1. mrr
2. ndcg
3. recall @k

In [10]:
import pandas as pd
import numpy as np
from litellm import completion
import json
from rich.pretty import pprint
from tqdm import tqdm
from rich.console import Console
from rich.panel import Panel

from pymupdf import Document as FitzDocument

from pymupdf4llm import IdentifyHeaders, to_markdown
import pymupdf
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry, OpenAIEmbeddings
import os
import requests
import tenacity
import time
import tiktoken
from dotenv import load_dotenv
load_dotenv()

def process_pdf(pdf_path,additional_info=None):
    """
    This function processes a PDF file and extracts markdown text and metadata for each page.

    Args:
        pdf_path (str): The path to the PDF file.
        additional_info (dict, optional): Additional metadata to include in the output. Defaults to None.

    Returns:
        tuple: A tuple containing two dictionaries:
            - pymupdf_markdown_by_page: A dictionary mapping page numbers to their markdown text.
            - pymupdf_metadata_by_page: A dictionary mapping page numbers to their metadata.
    """
    hdr_info = IdentifyHeaders(pdf_path)
    # print(hdr_info)
    doc: FitzDocument = pymupdf.open(pdf_path)
    pymupdf_markdown_by_page = {}
    pymupdf_metadata_by_page = {}

    for page in tqdm(doc, desc="Processing pages"):
        extra_info = {}
        extra_info.update(doc.metadata)
        extra_info["page"] = page.number + 1
        extra_info["total_pages"] = len(doc)
        extra_info["file_path"] = str(pdf_path)
        if additional_info:
            extra_info.update(additional_info)
        text = to_markdown(
                doc, pages=[page.number], hdr_info=hdr_info, write_images=False
            )
        pymupdf_markdown_by_page[page.number] = text
        pymupdf_metadata_by_page[page.number] = json.dumps(extra_info)
    return pymupdf_markdown_by_page, pymupdf_metadata_by_page

In [3]:
# pdf_path

In [4]:



pdf_url = 'https://maint.loc.gov/law/help/artificial-intelligence/regulation-artificial-intelligence.pdf'  # Replace with the actual link
pdf_path = os.path.join(os.getcwd(), 'downloaded_pdf.pdf')  # Define the path where the PDF will be saved

# Check if the PDF is already downloaded
if not os.path.exists(pdf_path):
    print(f"Downloading PDF from {pdf_url}...")
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded PDF to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")

# Process the single PDF
final_data = []
markdown_by_page, metadata_by_page = process_pdf(pdf_path)
for page_number, text in markdown_by_page.items():
    final_data.append(
        {
            "text": text,
            "metadata": metadata_by_page[page_number],
            "page_number": page_number,
            "file_path": pdf_path,
            "title": os.path.basename(pdf_path),
        }
    )

Downloaded PDF to /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf


Processing pages:   0%|          | 0/138 [00:00<?, ?it/s]

Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1)

Processing pages:   2%|▏         | 3/138 [00:00<00:04, 29.26it/s]

]
Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]


...

Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1

Processing pages:  99%|█████████▊| 136/138 [00:24<00:00,  7.77it/s]

]
Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1]
Processing /Users/jshah/Documents/GitHub/til/blogs/rag_evals/downloaded_pdf.pdf...
[                                        ] (0/1=======================================[========================================] (1/1)

Processing pages: 100%|██████████| 138/138 [00:25<00:00,  5.49it/s]

]


In [5]:
# visualizing a page 

console = Console()
page_number=50
console.print(Panel(markdown_by_page[page_number]))

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ the issue of ‘black box’ algorithms” and is “focused on placing the responsibility on companies to              │
│ prevent harm.”[85]                                                                                              │
│                                                                                                                 │
│ The Committee’s report states that                                                                              │
│                                                                                                                 │
│ he second group of rights relate to the right to object to automated decision-making and                        │
│ to access the logic behind it. In our view, these rights, again a response by the EU to                         │
│ emerging challenges from Big Data and AI, have a legitimate rationale. They are aimed at                        │
│ curbing harms due to prejudice and discrimination in output data owing to evaluative                            │
│ determinations without human review. The solution provided by this right is to simply                           │
│ involve a step of human review, which is not per se immune from prejudice. This is a                            │
│ change pertaining to the operational structure of an organisation. Such a change may be                         │
│ necessitated, provided it is carefully tailored to specific organisations and the nature of                     │
│ their processing activity. This, in our view, is better achieved through an accountability                      │
│ framework which requires certain data fiduciaries, which may be making evaluative                               │
│ decisions through automated means, to set up processes that weed out discrimination. This                       │
│ is a constituent element of privacy by design which should be implemented by entities                           │
│ proactively, audited periodically and monitored by the DPA in case there are examples of                        │
│ unlawful processing. At the same time, such a model does not entirely denude the                                │
│ individual of agency. If discrimination has ensued as a result of per se lawful, yet                            │
│ discriminatory automated processing, individuals are always at liberty to go to courts for                      │
│ breach of fiduciary duties. Thus, the interests underlying such rights, can be more                             │
│ efficaciously achieved by an ex ante accountability model.[86]                                                  │
│                                                                                                                 │
│ **B. Automated Vehicles**                                                                                       │
│                                                                                                                 │
│ The Seventh Schedule of the Constitution of India lists legislative subjects that are in the exclusive          │
│ or concurrent jurisdiction of the central government or state governments. The regulation of motor              │
│ vehicles in India appears to be under the concurrent jurisdiction of both the central and state                 │
│ governments.[87] Motor vehicle road safety is regulated by the central level by the Motor Vehicles              │
│ Act[88] and the Central Motor Vehicle Rules.[89] State governments have their own laws and policies.            │
│ The current Motor Vehicle Act does not appear to allow for automated vehicles (AVs) or AV                       │
│ testing. However, amending legislation, which was passed in the Lok Sabha (lower house of                       │
│ Parliament) on April 10, 2017, but is still pending be

In [5]:


model = OpenAIEmbeddings(name='text-embedding-3-small')
# Define the LanceModel for your table
class Corpus(LanceModel):
    text: str = model.SourceField()  
    metadata: str = None 
    page_number: int = None  
    title: str = None
    vector: Vector(1536) = model.VectorField()

# Create the table
db = lancedb.connect("./.lancedb")
tbl = db.create_table("corpus", schema=Corpus)





/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tbl.add(final_data)

In [7]:
tbl.head().to_pandas()

,text,metadata,page_number,title,vector
0,# Regulation of Artificial Intelligence in Sel...,"{""format"": ""PDF 1.6"", ""title"": ""Regulation of ...",0,downloaded_pdf.pdf,"[-0.016118292, 0.028076159, 0.04120163, 0.0225..."
1,This report is provided for reference purposes...,"{""format"": ""PDF 1.6"", ""title"": ""Regulation of ...",1,downloaded_pdf.pdf,"[0.023184152, 0.033128623, 0.099621244, 0.0307..."
2,### Contents\n\n**Comparative Summary ...........,"{""format"": ""PDF 1.6"", ""title"": ""Regulation of ...",2,downloaded_pdf.pdf,"[-0.033615343, -0.013138391, 0.054029632, 0.03..."
3,Taiwan ..........................................,"{""format"": ""PDF 1.6"", ""title"": ""Regulation of ...",3,downloaded_pdf.pdf,"[-0.06369742, -0.040109098, 0.06092408, -0.007..."
4,Norway ..........................................,"{""format"": ""PDF 1.6"", ""title"": ""Regulation of ...",4,downloaded_pdf.pdf,"[-0.038165003, -0.029677343, 0.06849071, 0.015..."


In [2]:
import instructor
import google.generativeai as genai
from pydantic import BaseModel, Field
import random

# Initialize the Gemini client
client = instructor.from_gemini(genai.GenerativeModel("gemini-1.5-flash-exp-0827"))

# Define the Pydantic model for the question-answer pair
class QuestionAnswerPair(BaseModel):
    question: str = Field(description="The generated question from the text chunk.")
    answer: str = Field(description="The answer to the generated question.")

# Sample text chunk (replace this with any random page or chunk of text)
text_chunk = """
The Reserve Bank of Australia (RBA) came into being on 14 January 1960 as Australia's central bank and banknote issuing authority, when the Reserve Bank Act 1959 removed the central banking functions from the Commonwealth Bank. The assets of the bank include the gold and foreign exchange reserves of Australia, which is estimated to have a net worth of A$101 billion. Nearly 94% of the RBA's employees work at its headquarters in Sydney, New South Wales and at the Business Resumption Site.
"""

# Function to generate synthetic questions
def generate_questions(chunk: str, num_questions: int = 5):
    questions = []
    for _ in range(num_questions):
        response = client.create(
            response_model=QuestionAnswerPair,
            messages=[
                {
                    "role": "system",
                    "content": "You are a world-class AI that excels at generating hypothetical search queries. "
                               "You're about to be given a text snippet and asked to generate a search query "
                               "which is specific to the text chunk you'll be given. Make sure to use information "
                               "from the text chunk."
                },
                {
                    "role": "user",
                    "content": f"Here is the text chunk: {chunk}"
                }
            ]
        )
        questions.append(response.question)
    return questions

# Generate and print synthetic questions
synthetic_questions = generate_questions(text_chunk,num_questions=1)
for i, question in enumerate(synthetic_questions, 1):
    print(f"Question {i}: {question}")

/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Question 1: ""What are the assets of the Reserve Bank of Australia?""


In [8]:
# Define the Pydantic model for the question-answer pair
client = instructor.from_gemini(genai.GenerativeModel("gemini-1.5-flash-exp-0827"))
class QuestionReasonPair(BaseModel):
    reasoning: str = Field(description="The step-by-step reasoning or chain of thought used to derive the answer.")
    question: str = Field(description="The generated question from the text chunk.")


@tenacity.retry(wait=tenacity.wait_fixed(5), stop=tenacity.stop_after_attempt(3))
def generate_questions(chunk: str):
    response = client.create(
        response_model=QuestionReasonPair,
        messages=[
            {
                "role": "system",
                "content": "<instructions>"
                           "Given a text sample or chunk, you must generate a single query that accurately represents "
                           "the main topic or information contained within the text. The query should be concise "
                           "and relevant to the content of the text."
                           "</instructions>"
                           "<important_instruction>"
                           "Ensure the query is specific and captures the essence of the text. Avoid overly broad "
                           "or vague queries."
                           "</important_instruction>"
                           "<steps>"
                           "<step>Read the text sample carefully to understand its main topic or information.</step>"
                           "<step>Identify the key points or themes in the text.</step>"
                           "<step>Formulate a chain of thought reasoning that leads to a query that encapsulates the main topic or key points of the text.</step>"
                           "<step>Generate a question based on the chain of thought reasoning.</step>"
                           "<step>Ensure the query is concise and relevant to the text.</step>"
                           "</steps>"
            },
            {
                "role": "user",
                "content": f"<text_sample>{chunk}</text_sample>"
            }
        ]
    )
    return response


In [5]:
db = lancedb.connect("./.lancedb")
table = db.open_table("corpus")
df = table.to_pandas()
random_row = df.sample(1)


In [10]:
output = generate_questions(random_row['text'].values[0])


In [13]:
#use panel and console and then print the output
console = Console()
console.print(Panel(output.reasoning))
console.print(Panel(output.question))

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ The text discusses the initiatives of Kazakhstan, Latvia and Lithuania regarding the adoption and integration   │
│ of AI within their economies and public sectors. The key initiatives in Kazakhstan include the establishment of │
│ tech parks, development of AI infrastructure, and designation of research centers.  Latvia focused on AI        │
│ integration in e-services, while Lithuania highlighted an innovation program that includes some AI elements. A  │
│ question that encapsulates the main topic would be what are the key strategies and initiatives that these       │
│ countries have for promoting the use of AI.                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ What are the key initiatives and strategies implemented by Kazakhstan and Baltic countries (Latvia and          │
│ Lithuania) to promote and integrate Artificial Intelligence (AI) within their respective economies and public   │
│ sectors?                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [18]:
# questions

In [11]:



num_of_questions = 50
questions = []
for i in tqdm(range(num_of_questions)):
    random_row = df.sample(1)
    try:
        output = generate_questions(random_row['text'].values[0])
        questions.append({"question": output.question,
                        "reasoning": output.reasoning, 
                        "page_number": random_row['page_number'].values[0]})
    except tenacity.RetryError as e:
        print(f"Failed to generate question after multiple retries: {e}")
    time.sleep(5)

  0%|          | 0/50 [00:00<?, ?it/s]

 28%|██▊       | 14/50 [01:25<03:39,  6.11s/it]

Failed to generate question after multiple retries: RetryError[<Future at 0x32717bed0 state=finished raised InstructorRetryException>]


 78%|███████▊  | 39/50 [04:24<01:13,  6.71s/it]

Failed to generate question after multiple retries: RetryError[<Future at 0x327360550 state=finished raised InstructorRetryException>]


100%|██████████| 50/50 [05:52<00:00,  7.04s/it]


In [14]:
len(questions)

50

In [15]:
# save this df to a table
pd.DataFrame(questions).to_csv('questions.csv',index=False)

In [16]:
df_questions = pd.read_csv('questions.csv')

In [17]:
df_questions.head()

,question,reasoning,page_number
0,How does China's AI development benefit from i...,The text discusses how Chinese AI companies ut...,137
1,What are the key goals and initiatives of the ...,The text discusses the Declaration of Cooperat...,71
2,What are the key initiatives and strategies im...,The text focuses on Germany's digital agenda a...,89
3,What are the key proposals and initiatives out...,The text sample focuses on Russia's approach t...,108
4,What are the countries included in the Europe ...,The provided text sample lists various countri...,3


In [19]:
def chunk_text_by_tokens(text, max_token_length=100, overlap=0):
    enc = tiktoken.get_encoding("o200k_base")
    tokens = enc.encode(text,disallowed_special=())
    chunks = []

    for i in range(0, len(tokens), max_token_length - overlap):
        chunk_tokens = tokens[i:i + max_token_length]
        chunk_text = enc.decode(chunk_tokens)
        chunks.append(chunk_text)

    return chunks

In [25]:
final_list = []
for index, row in df.iterrows():
    text = row['text']
    page_number = row['page_number']
    text_chunks = chunk_text_by_tokens(text,max_token_length=800,overlap=400)
    for text_chunk in text_chunks:
        final_list.append({'text': text_chunk, 'page_number': page_number})

In [23]:
len(final_list)

422

In [29]:
model = OpenAIEmbeddings(name='text-embedding-3-small')
# Define the LanceModel for your table
class Corpus(LanceModel):
    text: str = model.SourceField()  
    page_number: int = None  
    vector: Vector(1536) = model.VectorField()

# Create the table
db = lancedb.connect("./.lancedb")
tbl = db.create_table("eval_corpus", schema=Corpus)
tbl.add(final_list)


In [31]:
tbl.create_fts_index("text")

In [30]:


# Define a function to calculate MRR
def mean_reciprocal_rank(results, relevant_indices):
    reciprocal_ranks = []
    for result, relevant in zip(results, relevant_indices):
        for rank, doc in enumerate(result, start=1):
            if doc in relevant:
                reciprocal_ranks.append(1 / rank)
                break
        else:
            reciprocal_ranks.append(0)
    return np.mean(reciprocal_ranks)

# Define a function to calculate Recall@K
def recall_at_k(results, relevant_indices, k):
    recalls = []
    for result, relevant in zip(results, relevant_indices):
        relevant_set = set(relevant)
        retrieved_set = set(result[:k])
        recalls.append(len(relevant_set & retrieved_set) / len(relevant_set))
    return np.mean(recalls)

# Manual implementation of DCG (Discounted Cumulative Gain)
def dcg_at_k(relevance_scores, k):
    relevance_scores = np.asfarray(relevance_scores)[:k]
    if relevance_scores.size:
        return relevance_scores[0] + np.sum(relevance_scores[1:] / np.log2(np.arange(2, relevance_scores.size + 1)))
    return 0.

# Manual implementation of NDCG (Normalized Discounted Cumulative Gain)
def ndcg_at_k(results, relevant_indices, k):
    ndcg_scores = []
    for result, relevant in zip(results, relevant_indices):
        relevance = [1 if doc in relevant else 0 for doc in result[:k]]
        dcg = dcg_at_k(relevance, k)
        ideal_relevance = sorted(relevance, reverse=True)
        idcg = dcg_at_k(ideal_relevance, k)
        ndcg = dcg / idcg if idcg > 0 else 0
        ndcg_scores.append(ndcg)
    return np.mean(ndcg_scores)


In [45]:

def evaluate_search(tbl, df_questions, query_types=['fts', 'vector', 'hybrid']):
    metrics_list = []
    
    for query_type in query_types:
        print(f"Evaluating {query_type} search...")
        mrr_scores, ndcg_scores, recall_scores = [], [], []
        
        for index, row in df_questions.iterrows():
            query = row['question']
            page_number = row['page_number']
            
            search_results = tbl.search(query, query_type=query_type).limit(10).to_list()
            retrieved_pages = [result['page_number'] for result in search_results]
            relevant_pages = [page_number]
            
            mrr = mean_reciprocal_rank([retrieved_pages], [relevant_pages])
            ndcg = ndcg_at_k([retrieved_pages], [relevant_pages], k=10)
            recall = recall_at_k([retrieved_pages], [relevant_pages], k=10)
            
            mrr_scores.append(mrr)
            ndcg_scores.append(ndcg)
            recall_scores.append(recall)
            
            metrics_list.append({
                'method': query_type,
                'question': query,
                'page_number': page_number,
                'MRR': mrr,
                'NDCG@10': ndcg,
                'Recall@10': recall
            })
        
        avg_mrr = np.mean(mrr_scores)
        avg_ndcg = np.mean(ndcg_scores)
        avg_recall = np.mean(recall_scores)
        
        print(f"Average MRR: {avg_mrr:.4f}")
        print(f"Average NDCG@10: {avg_ndcg:.4f}")
        print(f"Average Recall@10: {avg_recall:.4f}")
        print()
    
    return pd.DataFrame(metrics_list)


In [46]:

# Run the evaluation
results_df = evaluate_search(tbl, df_questions)

# Display the first few rows of the results
print(results_df.head())

# Optional: Save the results to a CSV file
results_df.to_csv('search_evaluation_results.csv', index=False)

Evaluating fts search...
Average MRR: 0.7730
Average NDCG@10: 0.7833
Average Recall@10: 0.9000

Evaluating vector search...
Average MRR: 0.9467
Average NDCG@10: 0.8810
Average Recall@10: 0.9800

Evaluating hybrid search...
Average MRR: 0.9367
Average NDCG@10: 0.8696
Average Recall@10: 0.9800

  method                                           question  page_number  MRR  \
0    fts  How does China's AI development benefit from i...          137  1.0   
1    fts  What are the key goals and initiatives of the ...           71  1.0   
2    fts  What are the key initiatives and strategies im...           89  1.0   
3    fts  What are the key proposals and initiatives out...          108  1.0   
4    fts  What are the countries included in the Europe ...            3  1.0   

    NDCG@10  Recall@10  
0  1.000000        1.0  
1  1.000000        1.0  
2  0.678104        1.0  
3  1.000000        1.0  
4  1.000000        1.0  


In [71]:
rag = RAGPretrainedModel.from_pretrained("jinaai/jina-colbert-v1-en",)

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [72]:

rag = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

Error while downloading from https://cdn-lfs.huggingface.co/repos/a0/4c/a04cd86fe01b157b8cc5a51615eef5d3e9dce42c5569c2b74b15f66522dbba90/3f58890b1dfdfec066ef12ba431fa9d56992da9e30a53489242c4156e37e9017?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1726898591&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNjg5ODU5MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9hMC80Yy9hMDRjZDg2ZmUwMWIxNTdiOGNjNWE1MTYxNWVlZjVkM2U5ZGNlNDJjNTU2OWMyYjc0YjE1ZjY2NTIyZGJiYTkwLzNmNTg4OTBiMWRmZGZlYzA2NmVmMTJiYTQzMWZhOWQ1Njk5MmRhOWUzMGE1MzQ4OTI0MmM0MTU2ZTM3ZTkwMTc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=Dp8dlDh0ALbaOuy2JdI0cL09gKhwoR7sEorZIjgQfoUR-7VQz%7Ev6k0nyJr6WmoJCqWKMtN3n711g93%7EfF15yU3yj5DA%7EsrYCSGl2uX8tuxW4GmXdQxMdCo5p5RTgZcXUJ88agFfld3yrY9vYDQ9CoQ2JmIlIYO8OkTsbgsW5fpvt5WS1DyzaEJ6O9lXiETVCt2f3LsLjngFX6b1cnAYwWKC5DOplHVHftrMEdnh9amakuUar

In [66]:
from ragatouille import RAGPretrainedModel
def create_ragatouille_index(final_list, index_name="ragatouille_index", model="answerdotai/answerai-colbert-small-v1"):
    rag = RAGPretrainedModel.from_pretrained(model)
    
    # Create a list of documents with their text content
    docs = [item['text'] for item in final_list]
    
    # Create separate lists for document IDs and metadata
    doc_ids = [f"doc_{i}" for i in range(len(docs))]
    doc_metadata = [{'page_number': item['page_number']} for item in final_list]
    
    rag.index(
        collection=docs,
        document_ids=doc_ids,
        document_metadatas=doc_metadata,
        index_name=index_name,
        max_document_length=512,
        split_documents=False,  # We've already split the documents
    )
    return rag

# Create the index


In [74]:
rag_model_answerdotai = create_ragatouille_index(final_list, model="answerdotai/answerai-colbert-small-v1",index_name="rag_model_answerdotai")
rag_model_jina = create_ragatouille_index(final_list, model="jinaai/jina-colbert-v1-en",index_name="rag_model_jina")
rag_model_colbert_v2 = create_ragatouille_index(final_list, model="colbert-ir/colbertv2.0",index_name="rag_model_colbert_v2")


---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Sep 17, 23:18:22] #> Creating directory .ragatouille/colbert/indexes/rag_model_answerdotai 




/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


[Sep 17, 23:18:23] [0] 		 #> Encoding 422 passages..


  0%|          | 0/14 [00:00<?, ?it/s]/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/torch/amp/autocast_mode.py:265: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
100%|██████████| 14/14 [00:19<00:00,  1.41s/it]

[Sep 17, 23:18:43] [0] 		 avg_doclen_est = 277.8104248046875 	 len(local_sample) = 422
[Sep 17, 23:18:43] [0] 		 Creating 4,096 partitions.
[Sep 17, 23:18:43] [0] 		 *Estimated* 117,235 embeddings.
[Sep 17, 23:18:43] [0] 		 #> Saving the indexing plan to .ragatouille/colbert/indexes/rag_model_answerdotai/plan.json ..


...

[Sep 17, 23:24:33] [0] 		 #> Encoding 422 passages..

















100%|██████████| 14/14 [00:52<00:00,  3.73s/it]
1it [00:53, 53.07s/it]
100%|██████████| 1/1 [00:00<00:00, 1281.88it/s]

[Sep 17, 23:25:26] #> Optimizing IVF to store map from centroids to list of pids..
[Sep 17, 23:25:26] #> Building the emb2pid mapping..
[Sep 17, 23:25:26] len(emb2pid) = 117236



100%|██████████| 4096/4096 [00:00<00:00, 150206.51it/s]

[Sep 17, 23:25:26] #> Saved optimized IVF to .ragatouille/colbert/indexes/rag_model_colbert_v2/ivf.pid.pt
Done indexing!


In [75]:
def evaluate_ragatouille_search(rag_model, df_questions, model, k=10):
    metrics_list = []
    mrr_scores, ndcg_scores, recall_scores = [], [], []
    
    for index, row in df_questions.iterrows():
        query = row['question']
        actual_page_number = row['page_number']
        
        search_results = rag_model.search(query, k=k)
        # print(search_results)
        retrieved_pages = [result['document_metadata']['page_number'] for result in search_results]
        relevant_pages = [actual_page_number]
        
        mrr = mean_reciprocal_rank([retrieved_pages], [relevant_pages])
        ndcg = ndcg_at_k([retrieved_pages], [relevant_pages], k=k)
        recall = recall_at_k([retrieved_pages], [relevant_pages], k=k)
        
        mrr_scores.append(mrr)
        ndcg_scores.append(ndcg)
        recall_scores.append(recall)
        
        metrics_list.append({
            'method': f'ragatouille_{model}',
            'question': query,
            'page_number': actual_page_number,
            'MRR': mrr,
            'NDCG@10': ndcg,
            'Recall@10': recall
        })
    
    avg_mrr = np.mean(mrr_scores)
    avg_ndcg = np.mean(ndcg_scores)
    avg_recall = np.mean(recall_scores)
    
    print(f"Ragatouille Search Results for model {model}:")
    print(f"Average MRR: {avg_mrr:.4f}")
    print(f"Average NDCG@10: {avg_ndcg:.4f}")
    print(f"Average Recall@10: {avg_recall:.4f}")
    
    return pd.DataFrame(metrics_list)

# Run the evaluation


In [76]:
# Run the evaluation for both rag_models
ragatouille_results_model_1 = evaluate_ragatouille_search(rag_model_answerdotai, df_questions, model='answerdotai_colbert_small')
ragatouille_results_model_2 = evaluate_ragatouille_search(rag_model_jina, df_questions, model='jinaa_colbert_v1_en')
ragatouille_results_model_3 = evaluate_ragatouille_search(rag_model_colbert_v2, df_questions, model='colbert_v2.0')

# Combine the results from both models
combined_results = pd.concat([ragatouille_results_model_1, ragatouille_results_model_2, ragatouille_results_model_3])

# Display the first few rows of the combined results
print(combined_results.head())

# Optional: Save the combined results to a CSV file
combined_results.to_csv('ragatouille_evaluation_results_combined.csv', index=False)

Loading searcher for index rag_model_answerdotai for the first time... This may take a few seconds
[Sep 17, 23:25:28] #> Loading codec...
[Sep 17, 23:25:28] #> Loading IVF...
[Sep 17, 23:25:28] #> Loading doclens...


/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/colbert/search/index_loader.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ivf, ivf_lengths = torch.load

[Sep 17, 23:25:28] #> Loading codes and residuals...



  0%|          | 0/1 [00:00<?, ?it/s]/Users/jshah/micromamba/envs/hack/lib/python3.11/site-packages/colbert/indexing/codecs/residual_embeddings.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . How does China's AI development benefit from its partnerships with African nations, particularly Zimbabwe, in terms of data and technological advancement?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2129,  2515,  2859,  1005,  1055,  9932,  2458,  5770,
         2013,  2049, 13797,  2007,  3060,  3741,  1010,  3391, 11399,  1010,
         1999,  3408,  1997,  2951,  1998, 10660, 12607,  1029,   102,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0])



...

100%|██████████| 1/1 [00:00<00:00, 5084.00it/s]

[Sep 17, 23:25:37] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 158.93it/s]

Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . How does China's AI development benefit from its partnerships with African nations, particularly Zimbabwe, in terms of data and technological advancement?, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2129,  2515,  2859,  1005,  1055,  9932,  2458,  5770,
         2013,  2049, 13797,  2007,  3060,  3741,  1010,  3391, 11399,  1010,
         1999,  3408,  1997,  2951,  1998, 10660, 12607,  1029,   102,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0])



Ragatouille Search Results for model colbert_v2.0:
Average MRR: 0.8902
Average NDCG@10: 0.9032
Average Recall@10: 1.0000
                                  method  \
0  ragatouille_answerdotai_colbert_small   
1  ragatouille_answerdotai_colbert_small   
2  ragatouille_answerdotai_colbert_small   
3  ragatouille_answerdotai_colbert_small   
4  ragatouille_answerdotai_colbert_small   

                                            question  page_number       MRR  \
0  How does China's AI development benefit from i...          137  1.000000   
1  What are the key goals and initiatives of the ...           71  1.000000   
2  What are the key initiatives and strategies im...           89  1.000000   
3  What are the key proposals and initiatives out...          108  1.000000   
4  What are the countries included in the Europe ...            3  0.333333   

    NDCG@10  Recall@10  
0  1.000000        1.0  
1  1.000000        1.0  
2  0.815465        1.0  
3  1.000000        1.0  
4  0.630930   